# GAN - MNIST + Keras

## 0. Imports

In [29]:
# Maths
import numpy                                            as np

# Matplotlib
import matplotlib                                       as mp
import matplotlib.pyplot                                as pt

# Machine / Deep Learning
import tensorflow                                       as tf
import keras                                            as ks
from keras              import models                   as mls
from keras              import layers                   as lys
from keras.datasets     import mnist                    as mn
from keras.utils        import to_categorical           as tc

# Custom
from lib_monogan         import MonoGAN
from lib_monogan_monitor import MonoGAN_Monitor

# Versions
print( f"Numpy .... : {np.__version__}" )
print( f"Matplotlib : {mp.__version__}" )
print( f"Tensorflow : {tf.__version__}" )
print( f"Keras .... : {ks.__version__}" )

Numpy .... : 1.19.2
Matplotlib : 3.3.2
Tensorflow : 2.4.0
Keras .... : 2.4.3


## 1. Set Up Model

### 1.0 Set Up Discriminator

In [30]:
# Set up discriminator

dr = mls.Sequential( name = "Discriminator" )

dr.add( lys.Input( shape = ( 28, 28, 1 ) ) )
dr.add( lys.Conv2D( 64, ( 3, 3 ), strides = ( 2, 2 ), padding = "same" ) )
dr.add( lys.LeakyReLU( alpha = 0.2 ) )
dr.add( lys.Conv2D( 128, ( 3, 3 ), strides = ( 2, 2 ), padding = "same" ) )
dr.add( lys.LeakyReLU( alpha = 0.2 ) )
dr.add( lys.GlobalMaxPooling2D() )
dr.add( lys.Dense( 1 ) )

dr.summary()

Model: "Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 7, 7, 128)         0         
_________________________________________________________________
global_max_pooling2d_6 (Glob (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 74,625
Trainable params: 74,625
Non-trainable params: 0
_________________________________________________

### 1.1 Set Up Generator

In [31]:
dimension = 128

gr = mls.Sequential( name = "Generator" )

gr.add( lys.Input( shape = ( dimension, ) ) )
gr.add( lys.Dense( 7 * 7 * 128 ) )
gr.add( lys.LeakyReLU( alpha = 0.2 ) )
gr.add( lys.Reshape( ( 7, 7, 128 ) ) )
gr.add( lys.Conv2DTranspose( 128, ( 4, 4 ), strides = ( 2, 2 ), padding = "same" ) )
gr.add( lys.LeakyReLU( alpha = 0.2 ) )
gr.add( lys.Conv2DTranspose( 128, ( 4, 4 ), strides = ( 2, 2 ), padding = "same" ) )
gr.add( lys.LeakyReLU( alpha = 0.2 ) )
gr.add( lys.Conv2DTranspose( 1, ( 7, 7 ), padding = "same", activation = "sigmoid" ) )

gr.summary()

Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 6272)              809088    
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 6272)              0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_12 (Conv2DT (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_13 (Conv2DT (None, 28, 28, 128)       262272    
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 28, 28, 128)       0 

## 2. Setup MNIST Dataset 

In [32]:
( tr_images, tr_labels ), ( ts_images, ts_labels ) = mn.load_data()

batch_size = 64

all_digits = np.concatenate( [tr_images, ts_images] )
all_digits = all_digits.astype( 'float32' ) / 255
all_digits = np.reshape( all_digits, ( -1, 28, 28, 1 ) )

dataset = tf.data.Dataset.from_tensor_slices( all_digits )
dataset = dataset.shuffle( buffer_size=1024 ).batch( batch_size ).prefetch( 32 )

## 3. Create GAN Model

In [33]:
epochs = 30

gan = MonoGAN(
    discriminator   = dr,
    generator       = gr,
    dimension       = dimension
)

## 4.  Compile GAN Model

In [34]:
gan.compile(
    dr_opt          = ks.optimizers.Adam( learning_rate = 0.0003 ),
    gr_opt          = ks.optimizers.Adam( learning_rate = 0.0003 ),
    loss_function   = ks.losses.BinaryCrossentropy( from_logits = True )
)


## 4. Train Model

In [20]:
gan.fit(
    dataset,
    epochs = epochs,
    callbacks = [ MonoGAN_Monitor( image_number = 3, dimension = dimension ) ]
)

Epoch 1/5
938/938 [==============================] - 66s 69ms/step - loss: 0.8713 - accuracy: 0.8633
Epoch 2/5
938/938 [==============================] - 60s 64ms/step - loss: 0.0243 - accuracy: 0.9948
Epoch 3/5
938/938 [==============================] - 41s 43ms/step - loss: 0.0150 - accuracy: 0.9967
Epoch 4/5
938/938 [==============================] - 46s 49ms/step - loss: 0.0168 - accuracy: 0.9961
Epoch 5/5
938/938 [==============================] - 44s 47ms/step - loss: 0.0163 - accuracy: 0.9964


## 5. Evaluate Model

In [22]:
for i in range( 3 ):
    pt.figure( figsize = ( 30, 30 ) )
    im = pt.imread( f"./generated_img_{i}_29.png" )
    pt.imshow( im )

313/313 [==============================] - 2s 8ms/step - loss: 0.0508 - accuracy: 0.9910
0.9909999966621399
